In [3]:
import asyncio
import websockets
import json

async def test_websocket():
    uri = "ws://localhost:8001/ws/assistant/?api_key=chave_teste"
    async with websockets.connect(uri) as websocket:
        message = {"message": "Hello, server!"}
        await websocket.send(json.dumps(message))
        response = await websocket.recv()
        print(f"Recebeu do servidor: {response}")

await test_websocket()


Recebeu do servidor: {"message": "Hello, server!"}


In [9]:
import asyncio
import websockets
import json

async def test_cart_consumer():
    uri = "ws://localhost:8001/ws/cart/?token=token_unico"
    async with websockets.connect(uri) as websocket:

        comida_id = 10

        add_message = {
            "action": "add",
            "comida_id": comida_id,
            "quantidade": 2
        }
        await websocket.send(json.dumps(add_message))
        response = await websocket.recv()
        print(f"Resposta ao adicionar: {response}")

        view_message = {"action": "view"}
        await websocket.send(json.dumps(view_message))
        response = await websocket.recv()
        print(f"Itens no carrinho: {response}")

        remove_message = {
            "action": "remove",
            "comida_id": comida_id
        }
        
        await websocket.send(json.dumps(remove_message))
        response = await websocket.recv()
        print(f"Resposta ao remover: {response}")
        await websocket.close()

await test_cart_consumer()


Resposta ao adicionar: {"status": "item adicionado"}
Itens no carrinho: {"cart": [{"comida__comida": "Sushi", "quantidade": 1}]}
Resposta ao remover: {"status": "item removido"}
